# User guide

此文档将讲述如何正确地在电脑上安装并运行`Dynamic-DeepHit`。此程序包已在Windows 10环境下测试过并可以正常表现。

在开始一切前，请确保您满足以下条件：
- 拥有一台电脑
- 电脑至少有3.5GHz的主频和4GB的内存
- 良好的网络连接，以及网络代理/梯子

## 准备步骤

### 安装Python和虚拟环境

此程序包基于Python编程语言。因此，在开始之前，请在电脑上安装Python。这里我统一使用Python的Anaconda3发布，安装步骤如下：
- 访问Anaconda3官网
- 下载Anaconda3，并在电脑上安装

安装Anaconda3后，使用conda安装虚拟环境。


### 准备患者数据

患者的纵向生存数据必须被整理成一个2维的，最好为UTF-8编码格式的.csv文件。这些数据必须被保存在主路径下的`\data`文件夹中。

这些数据必须是“长格式”（long）的，每行（row）代表某位患者的某一次访视，每列（column）代表不同的数据（如甲胎蛋白，性别，年龄等）。举例：一位患者有三次访视数据，那么在数据库.csv文件中，这位患者的数据将有3行构成，每行各代表其第一，二和三次数据。

额外地，文件必须确保每位患者都拥有至少以下四种数据。这些数据最好一个缺失值也不要有，否则可能会未知地影响程序的运行。这些数据的列名必须与以下列出的名字完全相同（被硬编码入代码中）。

- ‘ID’：每位患者的独有编号。必须注意：这些编号必须是可以被转化成数值的！！！！！！比如，如果ID的值为'EA-01-0001'，则程序会报错；ID为‘1’‘2’则没有问题。附带的R程序中的自建方程toDummy()可以将字符形式的患者编号转化成数字。这个程序会将ID值相等的行归类为同一位患者的不同次访视。

- ‘Time’：此次访视的时间。每位患者的第一次访视（即所谓“基线数据”）的Time都应该等于0；第二及未来访视的Time值为此次访视时间与第一次访视时间的绝对差。例如：访视一为2021年3月1日；访视二为2021年5月1日；若选择Time单位为日，则Time_1 = 0, Time_2 = 51。
    - 为了确保不知名的bug出现，请尽量确保同一名患者不会出现两次时间相同的访视。
    - Time的值是此次访视时间与**基线访视时间**的差，不是与之前一次的！！！
    - 目前不清楚Time的单位的选择是否会对模型的性能有显著影响。目前，在PreCar和PBC队列中，使用月作为基本单位貌似都没有严重的问题。

- ‘Times’：患者的“生存时间”。显而易见地，与ID一样，同一名患者的不同次访视的Times值都应该一样。请确保没有将Times和Time搞混，且两者单位相同。

- ‘Status’：患者在生存时间末端的状态。一般来说，将健康/删失患者标记为0，转癌患者标记为1即可。

其他数据的处理：本代码自带一个将数据进行对数转换（log-transform）的方法。在params.json里声明你想对数转换的变量名称即可。注意，如果名称不出现在cont_list_in里，则不会进行对数转换。目前暂且不支持其它类型的数据转换。







### 最后检查

## 拟合模型

### 准备数据

### 开始拟合模型

完成前面的步骤后，若你想开始拟合模型，请在控制台输入并运行运行以下脚本：
```cmd
python Model_Training.py ./params/params.json
```

此脚本仅接受一个外部参数：`params.json`，储存了模型参数的JSON格式文件。关于此文件应如何编写，可以参考示例。
此脚本将在你的当前路径下创建一个文件夹，其名字与你的JSON文件中选择的训练集文件名字相同。在此文件夹下，每次重复运行代码都会重新生成一个新的路径，其中记录了此次运行拟合成的模型和其所有相关参数。


若想验证你的模型，请运行以下脚本：
```cmd
python Model_evaluation.py [data_mode] [(optional)keyword]
```
此脚本需要一个必要参数：`[data_mode]`，其指示脚本寻找由哪一个训练集拟合的模型：如，`PreCar`会让脚本前往当前路径下名为`PreCar`的文件夹。第二个参数为可选参数，若你键入了第二个参数，脚本会在`PreCar`文件夹下寻找文件夹名与第二个参数重叠的文件夹——第二个参数像是一个搜索关键词，指示我们寻找具体是哪一次的访视。若第二个参数空缺，模型会自动选择最近一次拟合的模型。